In [1]:
import sys
sys.path.append('..')

In [11]:
import pandas as pd
import numpy as np
from datetime import datetime
from constants import *
from eda_utils import *
from utils import eval_stat
import seaborn as sns
import pickle as pkl
import math

In [3]:
files = ['Bihar_536_Sensor_Data_Sep_2023_Screened.csv', 'Bihar_536_Sensor_Data_Oct_2023_Screened.csv', 'Bihar_536_Sensor_Data_Nov_2023_Screened.csv',
        'Bihar_512_Sensor_Data_May_Aug_Screened_Hourly.csv']

dataset = []

for f in files:
    dataset.append(pd.read_csv(f'{data_bihar}/{f}'))

/tmp/ipykernel_770988/1070597865.py:7: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,26

In [4]:
# dataset

In [5]:
lat_long_to_block = {}
lat_long_to_distr = {}

for data in dataset:
    cols = list(data.columns.values)[1:]

    for c in cols:
        lat, long = float(data.loc[2, c]), float(data.loc[3, c])

        if math.isnan(lat) or math.isnan(long):
            continue

        if (lat, long) not in lat_long_to_block:
            lat_long_to_block[(lat, long)] = data.loc[1, c]
            lat_long_to_distr[(lat, long)] = data.loc[0, c]

# print(lat_long_to_block)
# print(lat_long_to_distr)
print(len(lat_long_to_block), len(lat_long_to_distr))

536 536


In [6]:
# Mapping -> (Latitude, Longitude) -> {Timestamp -> ts, Device_ID -> d_id, RH -> rh, Temp -> temp, PM25 -> pm25}
full_data = []

for data in dataset:
    
    f_data = {}
    # f_data = {'timestamp': [], 'device_id': [], 'block': [], 'district': [], 'latitude': [], 'longitude': [], 'rh': [], 
    #              'temp': [], 'pm25': []}
    
    cols = list(data.columns.values)[1:]
    ts = list(data.loc[6:, data.columns.values[0]])

    for c in cols:
        lat, long = float(data.loc[2, c]), float(data.loc[3, c])

        if math.isnan(lat) or math.isnan(long):
            continue

        if (lat, long) not in f_data:
            f_data[(lat, long)] = {'timestamp': ts, 'block': lat_long_to_block[(lat, long)], 
                                   'district': lat_long_to_distr[(lat, long)], 'rh': [], 'temp': [], 'pm25': []}

        if c[0] == 'P':                 
            # f_data[(lat, long)]['device_id'] = c[5:]            # PM25_ is a 5 length substring
            f_data[(lat, long)]['pm25'] = data.loc[6:, c].to_list()
        elif c[0] == 'T':
            f_data[(lat, long)]['temp'] = data.loc[6:, c].to_list()
        elif c[0] == 'R':
            f_data[(lat, long)]['rh'] = data.loc[6:, c].to_list()
        
    full_data.append(f_data)

In [7]:
# full_data[0]

In [8]:
f_data = {'timestamp': [], 'block': [], 'district': [], 'latitude': [], 'longitude': [], 'rh': [], 'temp': [], 'pm25': []}

In [9]:
for data in full_data:

    for key in data:
        lat, long, sz = key[0], key[1], len(data[key]['timestamp'])
        
        assert len(data[key]['timestamp']) == len(data[key]['rh']) == len(data[key]['temp']) == len(data[key]['pm25']), "Improper logic"

        f_data['timestamp'].extend(data[key]['timestamp'])
        # f_data['device_id'].extend([data[key]['device_id']] * sz)
        f_data['block'].extend([data[key]['block']] * sz)
        f_data['district'].extend([data[key]['district']] * sz)
        f_data['latitude'].extend([lat] * sz)
        f_data['longitude'].extend([long] * sz)
        f_data['rh'].extend(data[key]['rh'])
        f_data['temp'].extend(data[key]['temp'])
        f_data['pm25'].extend(data[key]['pm25'])

In [10]:
# df_cols = ['timestamp', 'device_id', 'block', 'district', 'latitude', 'longitude', 'rh', 'temp', 'pm25']
# df_types = [np.datetime64, object, object, object, float, float, float, float, float]

### DataFrame initialization

In [11]:
df = pd.DataFrame(data=f_data)
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'pm25'])
df = df.sort_values(by='timestamp')
df

/tmp/ipykernel_770988/1143411847.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')


,timestamp,block,district,latitude,longitude,rh,temp,pm25
5728694,2023-05-01 00:00:00,TANKUPPA,GAYA,24.708,85.142,36.48166666666667,34.33166666666666,12.2
5554526,2023-05-01 00:00:00,MORWA,SAMASTIPUR,25.814,85.696,79.63833333333335,24.418333333333337,48.81666666666667
5643086,2023-05-01 00:00:00,BARUN,AURANGABAD,24.867,84.221,76.71333333333334,24.83833333333333,42.766666666666666
5834966,2023-05-01 00:00:00,UDWANTNAGAR,BHOJPUR,25.499,84.625,44.14761363636364,30.6664393939394,13.907196969696967
6133118,2023-05-01 00:00:00,KALER,ARWAL,25.123,84.500,77.19333333333333,24.56,48.699999999999996
...,...,...,...,...,...,...,...,...
4499149,2023-11-30 23:45:00,BASANTPUR.SI,SIWAN,26.170,84.663,100.0,NaN,538.866667
3686989,2023-11-30 23:45:00,KAKO,JEHANABAD,25.224,85.080,96.12,22.9,326.0
4018189,2023-11-30 23:45:00,BIND,NALANDA,25.310,85.686,85.76,23.673333,324.666667
4023949,2023-11-30 23:45:00,EKANGARSARAI,NALANDA,25.226,85.230,82.326667,24.28,295.533333


In [12]:
df.to_pickle(f'{data_bihar}/bihar_512_sensor_data.pkl')

In [3]:
df = pd.read_pickle(f'{data_bihar}/bihar_512_sensor_data_imputed.pkl')
df

,timestamp,block,district,latitude,longitude,rh,temp,pm25
5728694,2023-05-01 00:00:00,TANKUPPA,GAYA,24.708,85.142,36.481667,34.331667,12.2
5554526,2023-05-01 00:00:00,MORWA,SAMASTIPUR,25.814,85.696,79.638333,24.418333,48.81666666666667
5643086,2023-05-01 00:00:00,BARUN,AURANGABAD,24.867,84.221,76.713333,24.838333,42.766666666666666
5834966,2023-05-01 00:00:00,UDWANTNAGAR,BHOJPUR,25.499,84.625,44.147614,30.666439,13.907196969696967
6133118,2023-05-01 00:00:00,KALER,ARWAL,25.123,84.500,77.193333,24.560000,48.699999999999996
...,...,...,...,...,...,...,...,...
4499149,2023-11-30 23:45:00,BASANTPUR.SI,SIWAN,26.170,84.663,100.000000,16.519736,538.866667
3686989,2023-11-30 23:45:00,KAKO,JEHANABAD,25.224,85.080,96.120000,22.900000,326.0
4018189,2023-11-30 23:45:00,BIND,NALANDA,25.310,85.686,85.760000,23.673333,324.666667
4023949,2023-11-30 23:45:00,EKANGARSARAI,NALANDA,25.226,85.230,82.326667,24.280000,295.533333


In [13]:
df.count()

timestamp    3966470
block        3966470
district     3966470
latitude     3966470
longitude    3966470
rh           3845087
temp         3581239
pm25         3966470
dtype: int64

In [17]:
# times = list(df['timestamp'].unique())
# unique_dates = sorted(list(set([datetime.strptime(str(ts), "%Y-%m-%d %H:%M:%S").date() for ts in times])))
# unique_dates

In [17]:
df_new = df[['timestamp', 'latitude', 'longitude', 'rh', 'temp', 'pm25']].copy(deep=True)
df_new['timestamp'] = df_new['timestamp'].values.astype(float)
df_new

,timestamp,latitude,longitude,rh,temp,pm25
5728694,1.682899e+18,24.708,85.142,36.48166666666667,34.33166666666666,12.2
5554526,1.682899e+18,25.814,85.696,79.63833333333335,24.418333333333337,48.81666666666667
5643086,1.682899e+18,24.867,84.221,76.71333333333334,24.83833333333333,42.766666666666666
5834966,1.682899e+18,25.499,84.625,44.14761363636364,30.6664393939394,13.907196969696967
6133118,1.682899e+18,25.123,84.500,77.19333333333333,24.56,48.699999999999996
...,...,...,...,...,...,...
4499149,1.701388e+18,26.170,84.663,100.0,NaN,538.866667
3686989,1.701388e+18,25.224,85.080,96.12,22.9,326.0
4018189,1.701388e+18,25.310,85.686,85.76,23.673333,324.666667
4023949,1.701388e+18,25.226,85.230,82.326667,24.28,295.533333


In [18]:
data = df_new.to_numpy()
data.shape

(3966470, 6)

In [20]:
imputed_data = impute(data, method='iterative')

In [21]:
df['rh'] = imputed_data[:, 3]
df['temp'] = imputed_data[:, 4]
df.count()

timestamp    3966470
block        3966470
district     3966470
latitude     3966470
longitude    3966470
rh           3966470
temp         3966470
pm25         3966470
dtype: int64

In [22]:
df.to_pickle(f'{data_bihar}/bihar_512_sensor_data_imputed.pkl')

In [25]:
splits = ['random', 'timestamp', 'lat_long']

for split in splits:
    eval = train_and_eval(imputed_data, method='iterative', model_dir=model_dir, split=split, model_type='xgb')
    print(f'{split}: {eval}')

random: [{'Train_RMSE': 38.8930499962416, 'Train_Pearson_R': 0.8950592678039221, 'Test_RMSE': 39.168827828711386, 'Test_Pearson_R': 0.894671940402313}]
timestamp: [{'Train_RMSE': 38.79514724633832, 'Train_Pearson_R': 0.8966713472709734, 'Test_RMSE': 39.28663998907341, 'Test_Pearson_R': 0.8915623294754454}]
lat_long: [{'Train_RMSE': 38.79514724633832, 'Train_Pearson_R': 0.8966713472709734, 'Test_RMSE': 39.28663998907341, 'Test_Pearson_R': 0.8915623294754454}]
